# Objective
The purpose of this notebook is to create a unified dataset of landfast ice extent for the Alaskan coastline from the Mahoney / InteRFACE data collection as described in the exploratory data analysis notebooks and the summary documentation thereof. The notebook is based on the SNAP Data Pipeline concept and template.

## Pipeline Steps

The flow here is as follows:

    0. Setup - define directories, intial conditions, etc. Execute the setup code cell before any other step
    1. Fetch the data if it is currently not available locally
    2. Create a DateTime index that encompasses the entire range of the data and gracefully handle that the atomic units of data here have variable ranges or "durations" on the order of 20 days. Also check that the turnover between calendar years is handled.
    3. Map the existing data collection to this index.
    4. For each item in the existing data collection constrain the valid array values and establish a common and well known spatial reference.
    5. Create a merged raster from Chukchi and Beaufort data (or lackthereof) for each date in the time index with appropriate data types and NoData values.
    
Creating the ultimate GeoTIFFs (step 5) will require understanding the four possibles cases of data availability for each date in our DateTime Index.

    A: No data is available at all.
    B: A single raster exists from only one region.
    C: Multiple rasters exist from only one region.
    D: One or more rasters exist from both regions.

We can handle these cases like so:

 - Case A: Skip these dates for now, there is no data available. These dates can be brought back into a datacube at a later time if needed.

 - Case B: Take the single raster and merge it with the mask for whichever region has no data.

 - Case C: Take all the rasters and create a 3D array of values, and filter for the maximum of the time index. Then merge these data with the mask for whichever region has no data.

 - Case D. Same as above, but do the stacking and filtering for both regions, and then then merge those two outputs together.


## 0 - Setup

The following environment variables should be set before running this notebook:
- `$SRC_DIR`: the base directory for storing project data that should be backed up.
- `$DST_DIR`: the output directory where final products are placed.
- `$SCRATCH_DIR`: scratch directory for storing project data which does not need to be backed up.

I like to set these variables from within the notebook for clarity, but it could be done from your shell also.

In [2]:
# place all imports here
import config # this sets the GDAL and PROJ envs before importing geospatial libs
import os
import shutil
import re
import datetime
import pandas as pd
import rasterio as rio
import numpy as np
import pickle
import slurm
from pathlib import Path
from rasterio.crs import CRS
from rasterio.merge import merge
# local helper library, like clippy
import helpers as snappy
# constants
NCORES = 32
SLURM_MAIL = "cparr4@alaska.edu"
AC_CONDA_ENV = "/home/UA/cparr4/miniconda3/envs/conda-project"
COPY_SOURCE = False
project_dir = os.getcwd()


# set the environment variables and create directories and Path objects
os.environ["BASE_DIR"] = "/workspace/Data/Base_Data/Cryosphere/landfast_seaice"
src_p = Path(os.environ["BASE_DIR"])

os.environ["DST_DIR"] = "/atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/outputs"
dst_p = Path(os.environ["DST_DIR"]).mkdir(parents=True, exist_ok=True)
dst_p = Path(os.environ["DST_DIR"])

os.environ["SCRATCH_DIR"] = "/atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/scratch"
scratch_p = Path(os.environ["SCRATCH_DIR"]).mkdir(parents=True, exist_ok=True)
scratch_p = Path(os.environ["SCRATCH_DIR"])

# create scratch directory copied and renamed source GeoTIFFs
copy_dir = scratch_p.joinpath("src_copy")
copy_dir.mkdir(exist_ok=True, parents=True)

# create scratch directory for slurm scripts
slurm_dir = scratch_p.joinpath("slurm")
slurm_dir.mkdir(exist_ok=True, parents=True)

# create scratch directory for GeoTIFFs with corrected array values
arrfix_dir = scratch_p.joinpath("arrfix")
arrfix_dir.mkdir(exist_ok=True, parents=True)

# create scratch directory for regional mask or "dummy" GeoTIFFs
mask_dir = scratch_p.joinpath("mask")
mask_dir.mkdir(exist_ok=True, parents=True)

# create scratch directory for max-filtered GeoTIFFs to handle Cases C and D
max_dir = scratch_p.joinpath("max")
max_dir.mkdir(exist_ok=True, parents=True)

snappy.t_out = open("/dev/stdout", "w")
snappy.jprint("Executing pipeline Step 0 (setup)\n")
snappy.jprint(f"SRC_DIR set to {src_p}")
snappy.jprint(f"DST_DIR set to {dst_p}")
snappy.jprint(f"SCRATCH_DIR set to {scratch_p}\n")
snappy.jprint("Pipeline Step 0 (setup) complete\n")
snappy.jprint("Current snapshot of the working directories:")
_ = os.system("cd $SCRATCH_DIR/.. && tree -d")

Executing pipeline Step 0 (setup)

SRC_DIR set to /workspace/Data/Base_Data/Cryosphere/landfast_seaice
DST_DIR set to /atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/outputs
SCRATCH_DIR set to /atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/scratch

Pipeline Step 0 (setup) complete

Current snapshot of the working directories:
.
├── outputs
└── scratch
    ├── arrfix
    ├── mask
    ├── max
    ├── slurm
    └── src_copy

7 directories


## 1 - Fetch

We don't have to download any data - these are on our file system. We'll use the fetch stage to point at the data and create the list of filenames we'll use. We can also copy the source data to our scratch directory and do a file name clean up.

In [2]:
snappy.jprint("Executing pipeline Step 1 (fetch)\n")

slie_fps = [x for x in src_p.rglob("*_slie.tif")]
n_src_fps = len(slie_fps)
new_fps = [copy_dir / ''.join(x.parent.parent.name.lower() + "_" + x.name) for x in slie_fps]

if COPY_SOURCE:
    for src, dst in zip(slie_fps, new_fps):
        shutil.copy(src, dst)

snappy.jprint(f"Source data consists of {n_src_fps} Geotiffs, for example:")
snappy.jprint(slie_fps[0])

n_copied_fps = len([x for x in copy_dir.rglob("*_slie.tif")])

assert(n_src_fps == n_copied_fps)

snappy.jprint("Pipeline Step 1 complete\n")

Executing pipeline Step 1 (fetch)

Source data consists of 566 Geotiffs, for example:
/workspace/Data/Base_Data/Cryosphere/landfast_seaice/Beaufort/2000-01/r2000_286-311_slie.tif
Pipeline Step 1 complete



## 2 - Create Time Index

We'll write a few functions to parse file names and convert day-of-year (DOY) style dates to a more readable and DateTime friendly YYYY-MM-DD format. The file names are of the style `chukchi_r2007_326-350_slie.tif`. We'll construct a lookup where we can see all the date information associated with each file. We need to be aware that negative durations are lurking where we jump to the next calendar year.

Once those durations are fixed and all the date information is in place we can establish the start and end dates of the entire collection and create a DateTime index that encompasses all the data.

In [3]:
snappy.jprint("Executing pipeline Step 2 (time index)\n")

def create_regional_fp(fp):
    """Construct a new filename that contains the region info."""
    new_fp = dst_p / ''.join(fp.parent.parent.name.lower() + "_" + fp.name)
    return new_fp
    

def get_doy(fp):
    """Fetch the DOY range from a file name."""
    try:
        doy = re.match(r'.*([0-3][0-9][0-9]-[0-3][0-9][0-9])', fp).group(1)
    except:
        doy = re.match(r'.*([0-3][0-9][0-9]_[0-3][0-9][0-9])', fp).group(1)
        doy = doy.replace("_", "-")
    return doy


def get_re_year(fp):
    """Fetch a single year (YYYY) from a file name."""
    year = re.match(r'.*([1-3][0-9]{3})', fp).group(1)
    return int(year)


def split_doy(doy_range):
    """Split a DOY range string into a integer start and end."""
    doy_start, doy_end = doy_range.split("-")
    return int(doy_start), int(doy_end)    


def doy_date_to_YYYYMMDD(year, days):
    """Convert a a DOY + Year date to a YYYY-MM-DD datetime object."""
    dt = datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1)
    return dt


di = {}
for fp in slie_fps:
    
    k = create_regional_fp(fp).name
    di[k] = {}
    di[k]["start_year"] = get_re_year(k)
    di[k]["doy_range"] = get_doy(k)
    di[k]["doy_start"], di[k]["doy_end"] = split_doy(di[k]["doy_range"])
    di[k]["dt_start"] = doy_date_to_YYYYMMDD(di[k]["start_year"], di[k]["doy_start"])
    di[k]["dt_end"] = doy_date_to_YYYYMMDD(di[k]["start_year"], di[k]["doy_end"])
    di[k]["dt_duration"] = di[k]["dt_end"] - di[k]["dt_start"]

snappy.jprint("Example of a lookup with an incorrect duration:\n")
snappy.jprint(di['beaufort_r2000_349-003_slie.tif'])

def set_end_year(di):
    """Adds an end year for each file and handles rollover cases."""
    for k in di.keys():
        if di[k]["dt_duration"].days < 0:
            di[k]["end_year"] = di[k]["start_year"] + 1
            di[k]["dt_end"] = doy_date_to_YYYYMMDD(di[k]["end_year"], di[k]["doy_end"])
            di[k]["dt_duration"] = di[k]["dt_end"] - di[k]["dt_start"]

        else:
            di[k]["end_year"] = di[k]["start_year"]


set_end_year(di)
snappy.jprint("\n")
snappy.jprint("Example of a lookup with a corrected duration:\n")
snappy.jprint(di['beaufort_r2000_349-003_slie.tif'])


def get_first_dt(di):
    """Get the earliest chronological start date from the lookup."""
    start_dts = []
    for k in di.keys():
        start_dts.append(di[k]["dt_start"])
    start_dt = sorted(start_dts)[0]
    return start_dt


def get_last_dt(di):
    """Get the latest chronological end date from the lookup."""
    end_dts = []
    for k in di.keys():
        end_dts.append(di[k]["dt_end"])
    last_dt = sorted(end_dts)[-1]
    return last_dt

end = get_last_dt(di)
start = get_first_dt(di)
dt_range = pd.date_range(start=start, end=end)
snappy.jprint(f"\nEarliest chronological date is {start}\n")
snappy.jprint(f"Latest chronological date is {end}\n")
snappy.jprint("Pipeline Step 2 complete\n")

Executing pipeline Step 2 (time index)

Example of a lookup with an incorrect duration:

{'start_year': 2000, 'doy_range': '349-003', 'doy_start': 349, 'doy_end': 3, 'dt_start': datetime.datetime(2000, 12, 14, 0, 0), 'dt_end': datetime.datetime(2000, 1, 3, 0, 0), 'dt_duration': datetime.timedelta(days=-346)}


Example of a lookup with a corrected duration:

{'start_year': 2000, 'doy_range': '349-003', 'doy_start': 349, 'doy_end': 3, 'dt_start': datetime.datetime(2000, 12, 14, 0, 0), 'dt_end': datetime.datetime(2001, 1, 3, 0, 0), 'dt_duration': datetime.timedelta(days=20), 'end_year': 2001}

Earliest chronological date is 1996-10-17 00:00:00

Latest chronological date is 2008-07-14 00:00:00

Pipeline Step 2 complete



## 3 - Map Existing Data to the Time Index

Because each file spans many days, a single calendar day can be represented by more than one raster. We need to map dates to the representative rasters by checking if the date is in the time range represented by the file. We'll want to know how many files match from the Beaufort region, how many from the Chukchi region, and how many match total. We also expect that some calendar days will have no matches at all (e.g., August, which is not in the seasonal ice cycle). We'll create a new lookup keyed by each date in the DateTime Index that stores the above information and the matching file names.

Some individual days are represented by as many as six rasters across both regions! We'll stash some pickle files to indicate which dates correspond to which cases described in the introduction.

In [4]:
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

dt_di = {}

for dt in dt_range:
    dt_di[dt] = {}
    dt_di[dt]["matching data"] = []
    beaufort_count = 0
    chukchi_count = 0
    
    for k in di.keys():
        if time_in_range(di[k]["dt_start"], di[k]["dt_end"], dt):
            dt_di[dt]["matching data"].append(k)
            
            if "beaufort" in k.lower():
                beaufort_count += 1
            if "chukchi" in k.lower():
                chukchi_count += 1
            
    if len(dt_di[dt]["matching data"]) == 0:
        dt_di[dt]["matching data"].append("no data")
    
    dt_di[dt]["beaufort_count"] = beaufort_count
    dt_di[dt]["chukchi_count"] = chukchi_count
    dt_di[dt]["match_count"] = chukchi_count + beaufort_count
    
df = pd.DataFrame.from_dict(dt_di).T

snappy.jprint("\nSample of mapping data to a time stamp:\n")
snappy.jprint(df.sort_values("match_count", ascending=False).iloc[1])

conditions = [
    (df["match_count"] == 0),
    (df["match_count"] == 1),
    ((df["match_count"] > 1) & (df["chukchi_count"] * df["beaufort_count"] == 0)),
    ((df["match_count"] > 1) & (df["chukchi_count"] * df["beaufort_count"] != 0))]
choices = [1, 2, 3, 4] # these map to A, B, C, and D
df["merge_case"] = np.select(conditions, choices)

snappy.jprint("\nSummary of merge cases - most have data from both regions which is good.\n")
snappy.jprint(df["merge_case"].value_counts())
snappy.jprint("Writing pickles of each merge case to use in processing functions called by slurm.")

merge_case_b = df[df.merge_case == 2]
dt_arr_di_case_b = merge_case_b.T.to_dict()
with open(max_dir / "single_raster_single_region.pickle", "wb") as handle:
    pickle.dump(dt_arr_di_case_b, handle, protocol=pickle.HIGHEST_PROTOCOL)

merge_case_c = df[df.merge_case == 3]
dt_arr_di_case_c = merge_case_c.T.to_dict()
with open(max_dir / "many_raster_single_region.pickle", "wb") as handle:
    pickle.dump(dt_arr_di_case_c, handle, protocol=pickle.HIGHEST_PROTOCOL)

merge_case_d = df[df.merge_case == 4]
dt_arr_di_case_d = merge_case_d.T.to_dict()
with open(max_dir / "many_raster_both_regions.pickle", "wb") as handle:
    pickle.dump(dt_arr_di_case_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

snappy.jprint("\nPipeline Step 3 complete\n")


Sample of mapping data to a time stamp:

matching data     [beaufort_r2006_058-084_slie.tif, beaufort_r20...
beaufort_count                                                    3
chukchi_count                                                     3
match_count                                                       6
Name: 2006-03-23 00:00:00, dtype: object

Summary of merge cases - most have data from both regions which is good.

4    2510
3     829
1     797
2     153
Name: merge_case, dtype: int64
Writing pickles of each merge case to use in processing functions called by slurm.

Pipeline Step 3 complete



## 4 - Constrain and Fix the Array Values, Spatial References, and Generate Masks

The next step is to constrain our array values to a known range. We know from our EDA work the value schema in the existing collection is as follows:

 - 0: No landfast sea ice is present for this pixel. This means either water, or sea-ice that is NOT landfast.
 - 255: landfast sea ice is present for this pixel
 - 128: A landmask.
 - 63, 64, other oddball values: NoData

These values can be reduced to binary set where `0` indicates the absence of landfast sea ice and `1` the presence of it.

We'll define a function (see `fix_raster_values.py`) that forces this array value mapping and writes a new GeoTIFF.

Also, at this stage we can also create cohesive spatial references and raster creation profiles. Ultimately we want to create a merged raster as well, so we'll create three new rasters and raster profiles:
1. An empty mask (all zeros) for the Chukchi Region
2. An empty mask (all zeros) for the Beaufort Region
3. An empty mask (all zeros) for the combined Chukchi and Beaufort regions

Once we have established cohesive raster profiles, we can generate the function arguments that will be used to consolidate the array values. The fixed output GeoTIFFs will just have an "arrfix" prefix and be stashed in the `scratch/arrfix` directory.

In [10]:
snappy.jprint("Executing pipeline Step 4 (fix the array values and spatial references and generate masks)\n")

# create cohesive raster profiles
beauf_sample = new_fps[0]
chuk_sample = new_fps[-1]

with rio.open(beauf_sample) as beauf_src:
    beauf_src = rio.open(beauf_sample)
    beauf_profile = beauf_src.profile
beauf_profile["crs"] = CRS.from_epsg(3338)
beauf_profile.update(compress="lzw")

with rio.open(chuk_sample):
    chuk_src = rio.open(chuk_sample)
    chuk_profile = chuk_src.profile
chuk_profile["crs"] = CRS.from_epsg(3338)
chuk_profile.update(compress="lzw")

arr_merge, aff_merge = merge([beauf_src, chuk_src])

new_profile = chuk_profile.copy()
new_profile["height"], new_profile["width"] = arr_merge[0].shape
new_profile["transform"] = aff_merge
new_profile["crs"] = CRS.from_epsg(3338)
new_profile.update(compress="lzw")

with open(mask_dir / "beaufort_raster_profile.pickle", "wb") as handle:
    pickle.dump(beauf_profile, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(mask_dir / "chukchi_raster_profile.pickle", "wb") as handle:
    pickle.dump(chuk_profile, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(mask_dir / "both_region_profile.pickle", "wb") as handle:
    pickle.dump(new_profile, handle, protocol=pickle.HIGHEST_PROTOCOL)

# generate the masks and write to disk
with rio.open(mask_dir / "both_region_mask.tif", 'w', **new_profile) as dst:
        dst.write(np.zeros(arr_merge[0].shape), 1)

with rio.open(mask_dir / "beaufort_mask.tif", 'w', **beauf_profile) as dst:
        dst.write(np.zeros((beauf_profile["height"], beauf_profile["width"])), 1)

with rio.open(mask_dir / "chukchi_mask.tif", 'w', **chuk_profile) as dst:
        dst.write(np.zeros((chuk_profile["height"], chuk_profile["width"])), 1)
        
# we want the mask paths for later
both_mask_fp = mask_dir / "both_region_mask.tif"
chuk_mask_fp = mask_dir / "chukchi_mask.tif"
beauf_mask_fp = mask_dir / "beaufort_mask.tif"

# nice to indicate when data are written to disk
mask_fps = list(mask_dir.glob('*'))
snappy.jprint(f"{len(mask_fps)} files written to {mask_dir}")

snappy.jprint("\nGenerating Slurm scripts to correct the raster values and write new GeoTIFFs.")

# write slurm scripts to fix the raster values of an individual file
project_dir = os.getcwd()

chuk_fps = list(copy_dir.glob("chukchi*"))
chuk_pkl = mask_dir / "chukchi_raster_profile.pickle" 
beauf_fps = list(copy_dir.glob("beaufort*"))
beauf_pkl = mask_dir / "beaufort_raster_profile.pickle" 


chuk_slurm_fps = [
    slurm.write_sbatch(
        NCORES, SLURM_MAIL, slurm_dir, AC_CONDA_ENV, fp, chuk_pkl, project_dir, arrfix_dir
    ) 
    for fp in chuk_fps
]

snappy.jprint(f"\n{len(chuk_slurm_fps)} slurm scripts for the Chukchi written to {slurm_dir}")
snappy.jprint("\nExecuting Chukchi slurm scripts with sbatch...")

# Call the slurm scripts with the `sbatch` command
_ = [os.system(f"sbatch {fp}") for fp in chuk_slurm_fps]

snappy.jprint(f"\n{len(chuk_slurm_fps)} slurm scripts for the Beaufort written to {slurm_dir}")
snappy.jprint("\nExecuting Beaufort slurm scripts with sbatch...\n")
beaufort_slurm_fps = [
    slurm.write_sbatch(
        NCORES, SLURM_MAIL, slurm_dir, AC_CONDA_ENV, fp, beauf_pkl, project_dir, arrfix_dir
    ) 
    for fp in beauf_fps
]

_ = [os.system(f"sbatch {fp}") for fp in beaufort_slurm_fps]

snappy.jprint("Pipeline Step 4 complete\n")

Executing pipeline Step 4 (fix the array values and spatial references and generate masks)

6 files written to /atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/scratch/mask

Generating Slurm scripts to correct the raster values and write new GeoTIFFs.

316 slurm scripts for the Chukchi written to /atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/scratch/slurm

Executing Chukchi slurm scripts with sbatch...

316 slurm scripts for the Beaufort written to /atlas_scratch/cparr4/landfast_seaice/mahoney_preprocess/scratch/slurm

Executing Beaufort slurm scripts with sbatch...

Submitted batch job 3885672
Submitted batch job 3885673
Submitted batch job 3885674
Submitted batch job 3885675
Submitted batch job 3885676
Submitted batch job 3885677
Submitted batch job 3885678
Submitted batch job 3885679
Submitted batch job 3885680
Submitted batch job 3885681
Submitted batch job 3885682
Submitted batch job 3885683
Submitted batch job 3885684
Submitted batch job 3885685
Submitted batch

In [11]:
n_fixed_fps = len([x for x in arrfix_dir.rglob("*.tif")])
assert(n_copied_fps == n_fixed_fps)

## 5 - Create rasters that represent all possible data for a given date.

This where we handle cases B, C, and D that we described earlier. The code that executes the logic of these cases (also described earlier) lives in the Python scripts that are slurmed.

In [7]:
snappy.jprint("Executing pipeline Step 5 (Create merged rasters)\n")

case_b_pkl = Path(max_dir / "single_raster_single_region.pickle")
case_b_slurm_dts = [
    slurm.write_case_b(
        NCORES, SLURM_MAIL, slurm_dir, AC_CONDA_ENV, case_b_pkl, k, project_dir, mask_dir, arrfix_dir, dst_p
    ) 
    for k in dt_arr_di_case_b.keys()
]
_ = [os.system(f"sbatch {x}") for x in case_b_slurm_dts]

case_c_pkl = Path(max_dir / "many_raster_single_region.pickle")
case_c_slurm_dts = [
    slurm.write_case_c(
        NCORES, SLURM_MAIL, slurm_dir, AC_CONDA_ENV, case_c_pkl, k, project_dir, mask_dir, max_dir, arrfix_dir, dst_p
    ) 
    for k in dt_arr_di_case_c.keys()
]
_ = [os.system(f"sbatch {x}") for x in case_c_slurm_dts]

case_d_pkl = Path(max_dir / "many_raster_both_regions.pickle")
case_d_slurm_dts = [
    slurm.write_case_d(
        NCORES, SLURM_MAIL, slurm_dir, AC_CONDA_ENV, case_d_pkl, k, project_dir, mask_dir, max_dir, arrfix_dir, dst_p
    ) 
    for k in dt_arr_di_case_d.keys()
]
_ = [os.system(f"sbatch {x}") for x in case_d_slurm_dts]

snappy.jprint("Pipeline Step 5 complete\n")

Executing pipeline Step 5 (Create merged rasters)

Submitted batch job 3882180
Submitted batch job 3882181
Submitted batch job 3882182
Submitted batch job 3882183
Submitted batch job 3882184
Submitted batch job 3882185
Submitted batch job 3882186
Submitted batch job 3882187
Submitted batch job 3882188
Submitted batch job 3882189
Submitted batch job 3882190
Submitted batch job 3882191
Submitted batch job 3882192
Submitted batch job 3882193
Submitted batch job 3882194
Submitted batch job 3882195
Submitted batch job 3882196
Submitted batch job 3882197
Submitted batch job 3882198
Submitted batch job 3882199
Submitted batch job 3882200
Submitted batch job 3882201
Submitted batch job 3882202
Submitted batch job 3882203
Submitted batch job 3882204
Submitted batch job 3882205
Submitted batch job 3882206
Submitted batch job 3882207
Submitted batch job 3882208
Submitted batch job 3882209
Submitted batch job 3882210
Submitted batch job 3882211
Submitted batch job 3882212
Submitted batch job 38822

Submitted batch job 3882472
Submitted batch job 3882473
Submitted batch job 3882474
Submitted batch job 3882475
Submitted batch job 3882476
Submitted batch job 3882477
Submitted batch job 3882478
Submitted batch job 3882479
Submitted batch job 3882480
Submitted batch job 3882481
Submitted batch job 3882482
Submitted batch job 3882483
Submitted batch job 3882484
Submitted batch job 3882485
Submitted batch job 3882486
Submitted batch job 3882487
Submitted batch job 3882488
Submitted batch job 3882489
Submitted batch job 3882490
Submitted batch job 3882491
Submitted batch job 3882492
Submitted batch job 3882493
Submitted batch job 3882494
Submitted batch job 3882495
Submitted batch job 3882496
Submitted batch job 3882497
Submitted batch job 3882498
Submitted batch job 3882499
Submitted batch job 3882500
Submitted batch job 3882501
Submitted batch job 3882502
Submitted batch job 3882503
Submitted batch job 3882504
Submitted batch job 3882505
Submitted batch job 3882506
Submitted batch job 

Submitted batch job 3882766
Submitted batch job 3882767
Submitted batch job 3882768
Submitted batch job 3882769
Submitted batch job 3882770
Submitted batch job 3882771
Submitted batch job 3882772
Submitted batch job 3882773
Submitted batch job 3882774
Submitted batch job 3882775
Submitted batch job 3882776
Submitted batch job 3882777
Submitted batch job 3882778
Submitted batch job 3882779
Submitted batch job 3882780
Submitted batch job 3882781
Submitted batch job 3882782
Submitted batch job 3882783
Submitted batch job 3882784
Submitted batch job 3882785
Submitted batch job 3882786
Submitted batch job 3882787
Submitted batch job 3882788
Submitted batch job 3882789
Submitted batch job 3882790
Submitted batch job 3882791
Submitted batch job 3882792
Submitted batch job 3882793
Submitted batch job 3882794
Submitted batch job 3882795
Submitted batch job 3882796
Submitted batch job 3882797
Submitted batch job 3882798
Submitted batch job 3882799
Submitted batch job 3882800
Submitted batch job 

Submitted batch job 3883059
Submitted batch job 3883060
Submitted batch job 3883061
Submitted batch job 3883062
Submitted batch job 3883063
Submitted batch job 3883064
Submitted batch job 3883065
Submitted batch job 3883066
Submitted batch job 3883067
Submitted batch job 3883068
Submitted batch job 3883069
Submitted batch job 3883070
Submitted batch job 3883071
Submitted batch job 3883072
Submitted batch job 3883073
Submitted batch job 3883074
Submitted batch job 3883075
Submitted batch job 3883076
Submitted batch job 3883077
Submitted batch job 3883078
Submitted batch job 3883079
Submitted batch job 3883080
Submitted batch job 3883081
Submitted batch job 3883082
Submitted batch job 3883083
Submitted batch job 3883084
Submitted batch job 3883085
Submitted batch job 3883086
Submitted batch job 3883087
Submitted batch job 3883088
Submitted batch job 3883089
Submitted batch job 3883090
Submitted batch job 3883091
Submitted batch job 3883092
Submitted batch job 3883093
Submitted batch job 

Submitted batch job 3883352
Submitted batch job 3883353
Submitted batch job 3883354
Submitted batch job 3883355
Submitted batch job 3883356
Submitted batch job 3883357
Submitted batch job 3883358
Submitted batch job 3883359
Submitted batch job 3883360
Submitted batch job 3883361
Submitted batch job 3883362
Submitted batch job 3883363
Submitted batch job 3883364
Submitted batch job 3883365
Submitted batch job 3883366
Submitted batch job 3883367
Submitted batch job 3883368
Submitted batch job 3883369
Submitted batch job 3883370
Submitted batch job 3883371
Submitted batch job 3883372
Submitted batch job 3883373
Submitted batch job 3883374
Submitted batch job 3883375
Submitted batch job 3883376
Submitted batch job 3883377
Submitted batch job 3883378
Submitted batch job 3883379
Submitted batch job 3883380
Submitted batch job 3883381
Submitted batch job 3883382
Submitted batch job 3883383
Submitted batch job 3883384
Submitted batch job 3883385
Submitted batch job 3883386
Submitted batch job 

...

Submitted batch job 3884233
Submitted batch job 3884234
Submitted batch job 3884235
Submitted batch job 3884236
Submitted batch job 3884237
Submitted batch job 3884238
Submitted batch job 3884239
Submitted batch job 3884240
Submitted batch job 3884241
Submitted batch job 3884242
Submitted batch job 3884243
Submitted batch job 3884244
Submitted batch job 3884245
Submitted batch job 3884246
Submitted batch job 3884247
Submitted batch job 3884248
Submitted batch job 3884249
Submitted batch job 3884250
Submitted batch job 3884251
Submitted batch job 3884252
Submitted batch job 3884253
Submitted batch job 3884254
Submitted batch job 3884255
Submitted batch job 3884256
Submitted batch job 3884257
Submitted batch job 3884258
Submitted batch job 3884259
Submitted batch job 3884260
Submitted batch job 3884261
Submitted batch job 3884262
Submitted batch job 3884263
Submitted batch job 3884264
Submitted batch job 3884265
Submitted batch job 3884266
Submitted batch job 3884267
Submitted batch job 

Submitted batch job 3884526
Submitted batch job 3884527
Submitted batch job 3884528
Submitted batch job 3884529
Submitted batch job 3884530
Submitted batch job 3884531
Submitted batch job 3884532
Submitted batch job 3884533
Submitted batch job 3884534
Submitted batch job 3884535
Submitted batch job 3884536
Submitted batch job 3884537
Submitted batch job 3884538
Submitted batch job 3884539
Submitted batch job 3884540
Submitted batch job 3884541
Submitted batch job 3884542
Submitted batch job 3884543
Submitted batch job 3884544
Submitted batch job 3884545
Submitted batch job 3884546
Submitted batch job 3884547
Submitted batch job 3884548
Submitted batch job 3884549
Submitted batch job 3884550
Submitted batch job 3884551
Submitted batch job 3884552
Submitted batch job 3884553
Submitted batch job 3884554
Submitted batch job 3884555
Submitted batch job 3884556
Submitted batch job 3884557
Submitted batch job 3884558
Submitted batch job 3884559
Submitted batch job 3884560
Submitted batch job 

Submitted batch job 3884819
Submitted batch job 3884820
Submitted batch job 3884821
Submitted batch job 3884822
Submitted batch job 3884823
Submitted batch job 3884824
Submitted batch job 3884825
Submitted batch job 3884826
Submitted batch job 3884827
Submitted batch job 3884828
Submitted batch job 3884829
Submitted batch job 3884830
Submitted batch job 3884831
Submitted batch job 3884832
Submitted batch job 3884833
Submitted batch job 3884834
Submitted batch job 3884835
Submitted batch job 3884836
Submitted batch job 3884837
Submitted batch job 3884838
Submitted batch job 3884839
Submitted batch job 3884840
Submitted batch job 3884841
Submitted batch job 3884842
Submitted batch job 3884843
Submitted batch job 3884844
Submitted batch job 3884845
Submitted batch job 3884846
Submitted batch job 3884847
Submitted batch job 3884848
Submitted batch job 3884849
Submitted batch job 3884850
Submitted batch job 3884851
Submitted batch job 3884852
Submitted batch job 3884853
Submitted batch job 

Submitted batch job 3885112
Submitted batch job 3885113
Submitted batch job 3885114
Submitted batch job 3885115
Submitted batch job 3885116
Submitted batch job 3885117
Submitted batch job 3885118
Submitted batch job 3885119
Submitted batch job 3885120
Submitted batch job 3885121
Submitted batch job 3885122
Submitted batch job 3885123
Submitted batch job 3885124
Submitted batch job 3885125
Submitted batch job 3885126
Submitted batch job 3885127
Submitted batch job 3885128
Submitted batch job 3885129
Submitted batch job 3885130
Submitted batch job 3885131
Submitted batch job 3885132
Submitted batch job 3885133
Submitted batch job 3885134
Submitted batch job 3885135
Submitted batch job 3885136
Submitted batch job 3885137
Submitted batch job 3885138
Submitted batch job 3885139
Submitted batch job 3885140
Submitted batch job 3885141
Submitted batch job 3885142
Submitted batch job 3885143
Submitted batch job 3885144
Submitted batch job 3885145
Submitted batch job 3885146
Submitted batch job 

Submitted batch job 3885405
Submitted batch job 3885406
Submitted batch job 3885407
Submitted batch job 3885408
Submitted batch job 3885409
Submitted batch job 3885410
Submitted batch job 3885411
Submitted batch job 3885412
Submitted batch job 3885413
Submitted batch job 3885414
Submitted batch job 3885415
Submitted batch job 3885416
Submitted batch job 3885417
Submitted batch job 3885418
Submitted batch job 3885419
Submitted batch job 3885420
Submitted batch job 3885421
Submitted batch job 3885422
Submitted batch job 3885423
Submitted batch job 3885424
Submitted batch job 3885425
Submitted batch job 3885426
Submitted batch job 3885427
Submitted batch job 3885428
Submitted batch job 3885429
Submitted batch job 3885430
Submitted batch job 3885431
Submitted batch job 3885432
Submitted batch job 3885433
Submitted batch job 3885434
Submitted batch job 3885435
Submitted batch job 3885436
Submitted batch job 3885437
Submitted batch job 3885438
Submitted batch job 3885439
Submitted batch job 

In [19]:
n_expected_outputs = len(dt_arr_di_case_d.keys()) + len(dt_arr_di_case_c.keys()) + len(dt_arr_di_case_b.keys())
n_output_fps = len([x for x in dst_p.rglob("*.tif")])

assert(n_expected_outputs == n_output_fps)

# Pipeline Complete!